In [1]:
all_brands = set(['fujifilm', 'digital blue', 'bell howell', 'bell+howell', 'hello kitty', 'vistaquest', 'blackmagic', 'superheadz', 'hasselblad', 'hikivision', 'fuijifilm', 'panasonic', 'fugi film', 'fiji film', 'hikvision', 'polaroid', 'insignia', 'keychain', 'yourdeal', 'lowrance', 'logitech', 'fijifilm', 'bushnell', 'fujufilm', 'fugifilm', 'sylvania', 'olympul', 'lowepro', 'toshiba', 'olympus', 'samsung', 'finepix', 'fuifilm', 'samyang', 'aquapix', 'vivitar', 'neopine', 'minolta', 'easypix', 'sandisk', 'vivicam', 'philips', 'coleman', 'yashica', 'coolpix', 'emerson', 'sealife', 'crayola', 'intova', 'tamron', 'cannon', 'garmin', 'pentax', 'barbie', 'konica', 'mustek', 'aiptek', 'keedox', 'contax', 'konika', 'go pro', 'olymus', 'philip', 'wopson', 'pextax', 'wespro', 'disney', 'rollei', 'sakar', 'lytro', 'apple', 'lexar', 'kodak', 'vibe ', 'drift', 'vtech', 'vizio', 'kinon', 'haier', 'epson', 'gopro', 'croco', 'minox', 'nokia', 'dahua', 'sanyo', 'intel', 'kodax', 'ricoh', 'argus', 'lumix', 'nikon', 'absee', 'nicon', 'bmpcc', 'canon', 'casio', 'cobra', 'leica', 'sigma', 'sony', 'fuji', 'b h ', 'lego', 'benq', 'asus', 'pov', 'jvc', 'b+w', 'ion', 'lg ', 'dji', 'tvc', 'eos', 'ge ', 'vpc', 'dxg', 'svp', 'hp'])

In [2]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

w2v = Word2Vec.load("word2vec.model")

In [3]:
from os import listdir
from os.path import isfile, join
import json
import math

sentences = []

webs = [f for f in listdir('/home/sunji/EM_sigmod/2013_camera_specs')]
for w in webs:
    items = [f for f in listdir('/home/sunji/EM_sigmod/2013_camera_specs/'+w)]
    for it in items:
        with open('/home/sunji/EM_sigmod/2013_camera_specs/'+w+'/'+it, 'r') as f:
            data = json.load(f)
            sentence = ''
            for k, v in data.items():
                sentence += str(k)
                sentence += ' '
                if type(v) == list:
                    for vv in v:
                        sentence += str(vv)
                        sentence += ' '
                else:
                    sentence += str(v)
                    sentence += ' '
            sentences.append(sentence)

total_words_num = 0
            
word_freqs = {}
for s in sentences:
    for ss in s.split(' '):
        total_words_num += 1
        if ss.lower() in word_freqs:
            word_freqs[ss.lower()] += 1
        else:
            word_freqs[ss.lower()] = 1
for k in word_freqs.keys():
    word_freqs[k] /= float(total_words_num)

word_df = {}
for s in sentences:
    for ss in set(s.lower().split(' ')):
        if ss in word_df:
            word_df[ss] += 1
        else:
            word_df[ss] = 1
for k in word_df.keys():
    word_df[k] = math.log(len(sentences) / float(word_df[k]))
    
word_tfidf = {}
for k in word_df.keys():
    word_tfidf[k] = word_freqs[k] * word_df[k]
    

In [ ]:
sorted(word_tfidf.items(), key=lambda x:x[1], reverse=False)

In [4]:
import pandas as pd
structured_data = pd.read_csv('total_with_key_type.csv')
structured_data = structured_data.set_index('spec_id')

In [ ]:
candidate_pairs = pd.read_csv('/home/sunji/EM_sigmod/quickstart_package/candidate_pairs_quickstart.csv')

In [ ]:
len(structured_data[pd.isnull(structured_data['type'])])

In [ ]:
import pandas as pd
import swifter
products_labeled = pd.read_csv('/home/sunji/EM_sigmod/sigmod_large_labelled_dataset.csv')
# products_labeled['type'] = products_labeled['type'].fillna('')
# products_labeled['camera_title_overlap_freq'] = products_labeled['camera_title_overlap_freq'].fillna('')
# products_labeled['camera_title_unmatch_freq'] = products_labeled['camera_title_unmatch_freq'].fillna('')
def training_bloking(row):
    llll = structured_data.loc[row['left_spec_id']]['blocking_key']
    rrrr = structured_data.loc[row['right_spec_id']]['blocking_key']
    return llll == rrrr

products_labeled = products_labeled[products_labeled.swifter.apply(lambda x: training_bloking(x), axis=1)]


In [ ]:
import re
import editdistance

def feature_extraction(pairs_df):
    stopped_words = set(['digital camera', 'camera']) | all_brands

    camera_type_jaccard_dis = []
    camera_type_numeric_jaccard_dis = []
    camera_title_jaccard_dis = []
    camera_type_overlap_dis = []
    camera_type_numeric_overlap_dis = []
    camera_title_overlap_dis = []
    camera_title_overlap_freq = []
    camera_title_unmatch_freq = []
    camera_type_edit_dis = []
    camera_title_edit_dis = []
    camera_title_numeric_jaccard_dis = []
    camera_title_numeric_edit_dis = []
    
    cnt = 0
    for _, row in pairs_df.iterrows():
    #   camera_brand_dis
        if cnt % 100 == 0:
            print (cnt)
        cnt += 1
        llll = structured_data.loc[row['left_spec_id']]
        rrrr = structured_data.loc[row['right_spec_id']]

        lset = set(str(llll['type']).split(';'))
        rset = set(str(rrrr['type']).split(';'))
        
        max_len = max(len(str(llll['type'])), len(str(rrrr['type'])))
        camera_type_edit_dis.append(editdistance.eval(''.join(sorted(lset)), ''.join(sorted(rset))) / float(max_len))

        if len(lset) == 0 and len(rset) == 0:
            camera_type_jaccard_dis.append(0.0)
            camera_type_overlap_dis.append(0.0)
        else:
            camera_type_jaccard_dis.append(1.0 - len(lset&rset) / float(len(lset|rset)))
            camera_type_overlap_dis.append(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))

        lset = set(re.findall(r"\d+\.?\d*",str(llll['type'])))
        rset = set(re.findall(r"\d+\.?\d*",str(rrrr['type'])))

        if len(lset) == 0 and len(rset) == 0:
            camera_type_numeric_jaccard_dis.append(0.0)
            camera_type_numeric_overlap_dis.append(0.0)
        else:
            camera_type_numeric_jaccard_dis.append(1-len(lset&rset) / float(len(lset|rset)))
            camera_type_numeric_overlap_dis.append(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))

        lnumtitle = re.findall(r"\d+",str(llll['page_title']))
        rnumtitle = re.findall(r"\d+",str(rrrr['page_title']))
        
        lset = set(lnumtitle)
        rset = set(rnumtitle)
        
        if len(lset) == 0 and len(rset) == 0:
            camera_title_numeric_jaccard_dis.append(0.0)
            camera_title_numeric_edit_dis.append(0.0)
        else:
            camera_title_numeric_jaccard_dis.append(1.0 - len(lset&rset) / float(len(lset|rset)))
            max_len = max(len(lnumtitle), len(rnumtitle))
            camera_title_numeric_edit_dis.append(editdistance.eval(lnumtitle, rnumtitle) / float(max_len))
        
        ltitle = str(llll['page_title']).split(' ')
        rtitle = str(rrrr['page_title']).split(' ')
        lset = set(ltitle)
        rset = set(rtitle)

        if len(lset) == 0 and len(rset) == 0:
            camera_title_jaccard_dis.append(0.0)
            camera_title_overlap_dis.append(0.0)
            camera_title_edit_dis.append(0.0)
        else:
            camera_title_jaccard_dis.append(1.0-len(lset&rset) / float(len(lset|rset)))
            camera_title_overlap_dis.append(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))
            max_len = max(len(ltitle), len(rtitle))
            camera_title_edit_dis.append(editdistance.eval(ltitle, rtitle) / float(max_len))
        overlap_freqs = []
        for w in lset&rset:
            if w in word_tfidf:
                overlap_freqs.append(str(word_tfidf[w]))
        camera_title_overlap_freq.append(';'.join(overlap_freqs))
        unmatch_freqs = []
        for w in (lset|rset)-(lset&rset):
            if w in word_tfidf:
                unmatch_freqs.append(str(word_tfidf[w]))
        camera_title_unmatch_freq.append(';'.join(unmatch_freqs))


    pairs_df['camera_type_jaccard_dis'] = camera_type_jaccard_dis
    pairs_df['camera_type_overlap_dis'] = camera_type_overlap_dis
    pairs_df['camera_type_edit_dis'] = camera_type_edit_dis
    pairs_df['camera_type_numeric_jaccard_dis'] = camera_type_numeric_jaccard_dis
    pairs_df['camera_type_numeric_overlap_dis'] = camera_type_numeric_overlap_dis
    pairs_df['camera_title_jaccard_dis'] = camera_title_jaccard_dis
    pairs_df['camera_title_overlap_dis'] = camera_title_overlap_dis
    pairs_df['camera_title_edit_dis'] = camera_title_edit_dis
    pairs_df['camera_title_numeric_jaccard_dis'] = camera_title_numeric_jaccard_dis
    pairs_df['camera_title_numeric_edit_dis'] = camera_title_numeric_edit_dis
    pairs_df['camera_title_overlap_freq'] = camera_title_overlap_freq
    pairs_df['camera_title_unmatch_freq'] = camera_title_unmatch_freq
   
    
        

In [ ]:
feature_extraction(products_labeled)

In [ ]:
products_labeled

In [ ]:
products_labeled.to_csv('products_labeled_with_distance.csv', index=False)

In [ ]:
import pandas as pd
products_labeled = pd.read_csv('products_labeled_with_distance.csv')
products_labeled['camera_title_overlap_freq'] = products_labeled['camera_title_overlap_freq'].fillna('')
products_labeled['camera_title_unmatch_freq'] = products_labeled['camera_title_unmatch_freq'].fillna('')

In [ ]:
products_labeled = products_labeled.drop_duplicates(subset=['camera_type_jaccard_dis',
       'camera_type_overlap_dis', 'camera_type_edit_dis',
       'camera_type_numeric_jaccard_dis', 'camera_type_numeric_overlap_dis',
       'camera_title_jaccard_dis', 'camera_title_overlap_dis',
       'camera_title_edit_dis', 'camera_title_overlap_freq',
       'camera_title_unmatch_freq', 'camera_title_numeric_jaccard_dis', 'camera_title_numeric_edit_dis'], keep='first')

In [ ]:
xxxx = products_labeled[products_labeled.duplicated()]
xxx = xxxx.sort_values(by=['camera_type_jaccard_dis',
       'camera_type_overlap_dis', 'camera_type_edit_dis',
       'camera_type_numeric_jaccard_dis', 'camera_type_numeric_overlap_dis',
       'camera_title_jaccard_dis', 'camera_title_overlap_dis',
       'camera_title_edit_dis', 'camera_title_overlap_freq',
       'camera_title_unmatch_freq', 'camera_title_numeric_jaccard_dis', 'camera_title_numeric_edit_dis'])


In [ ]:
len(xxx)

In [ ]:
len(products_labeled[products_labeled['label'] == 0])


In [ ]:
# train_features = list(products_labeled[products_labeled['label'] == 1].iloc[0:40000, 3:11].values) + list(products_labeled[products_labeled['label'] == 0].iloc[0:100000, 3:11].values)
# train_overlap_freqs = list(products_labeled[products_labeled['label'] == 1].iloc[0:40000, 11:12].values) + list(products_labeled[products_labeled['label'] == 0].iloc[0:100000, 11:12].values)
# train_unmatch_freqs = list(products_labeled[products_labeled['label'] == 1].iloc[0:40000, 12:13].values) + list(products_labeled[products_labeled['label'] == 0].iloc[0:100000, 12:13].values)
# train_targets = list(products_labeled[products_labeled['label'] == 1].iloc[0:40000, 2:3].values) + list(products_labeled[products_labeled['label'] == 0].iloc[0:100000, 2:3].values)

# test_features = list(products_labeled[products_labeled['label'] == 1].iloc[40000:, 3:11].values) + list(products_labeled[products_labeled['label'] == 0].iloc[100000:, 3:11].values)
# test_overlap_freqs = list(products_labeled[products_labeled['label'] == 1].iloc[40000:, 11:12].values) + list(products_labeled[products_labeled['label'] == 0].iloc[100000:, 11:12].values)
# test_unmatch_freqs = list(products_labeled[products_labeled['label'] == 1].iloc[40000:, 12:13].values) + list(products_labeled[products_labeled['label'] == 0].iloc[100000:, 12:13].values)
# test_targets = list(products_labeled[products_labeled['label'] == 1].iloc[40000:, 2:3].values) + list(products_labeled[products_labeled['label'] == 0].iloc[100000:, 2:3].values)

train_features = list(products_labeled[products_labeled['label']==1].iloc[:40000, 3:13].values) + list(products_labeled[products_labeled['label']==0].iloc[:100000, 3:13].values)
train_overlap_freqs = list(products_labeled[products_labeled['label']==1].iloc[:40000, 13:14].values) + list(products_labeled[products_labeled['label']==0].iloc[:100000, 13:14].values)
train_unmatch_freqs = list(products_labeled[products_labeled['label']==1].iloc[:40000, 14:15].values) + list(products_labeled[products_labeled['label']==0].iloc[:100000, 14:15].values)
train_targets = list(products_labeled[products_labeled['label']==1].iloc[:40000, 2:3].values) + list(products_labeled[products_labeled['label']==0].iloc[:100000, 2:3].values)

test_features = list(products_labeled[products_labeled['label']==1].iloc[40000:, 3:13].values) + list(products_labeled[products_labeled['label']==0].iloc[100000:, 3:13].values)
test_overlap_freqs = list(products_labeled[products_labeled['label']==1].iloc[40000:, 13:14].values) + list(products_labeled[products_labeled['label']==0].iloc[100000:, 13:14].values)
test_unmatch_freqs = list(products_labeled[products_labeled['label']==1].iloc[40000:, 14:15].values) + list(products_labeled[products_labeled['label']==0].iloc[100000:, 14:15].values)
test_targets = list(products_labeled[products_labeled['label']==1].iloc[40000:, 2:3].values) + list(products_labeled[products_labeled['label']==0].iloc[100000:, 2:3].values)


train_loader = torch.utils.data.DataLoader(
        ProductsDataset(train_features, train_targets, train_overlap_freqs, train_unmatch_freqs), batch_size=256, shuffle=True)
test_loader = torch.utils.data.DataLoader(
        ProductsDataset(test_features, test_targets, test_overlap_freqs, test_unmatch_freqs), batch_size=1024, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np
import torch.utils.data

from torch.utils.data import DataLoader
import time

sequence_len = 1000
sequence_len_title = 20
class Dataset(object):
    """An abstract class representing a Dataset.

    All other datasets should subclass it. All subclasses should override
    ``__len__``, that provides the size of the dataset, and ``__getitem__``,
    supporting integer indexing in range from 0 to len(self) exclusive.
    """

    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError


def generate_embeddings(words, seq_len):
    embedding = []
    for word in words:
        if word in w2v:
            embedding.append(w2v[word])
#     print ('length: ', len(embedding), len(words))
    if len(embedding) < seq_len:
        if len(embedding) > 0:
            mask = np.pad([[1] for _ in range(len(embedding))], ((0, seq_len-len(embedding)), (0,0)), 'constant')
            embedding = np.pad(embedding, ((0, seq_len-len(embedding)), (0,0)), 'constant')
        else:
            mask = np.pad([[0]], ((0, seq_len-1), (0,0)), 'constant')
            embedding = np.pad([[0 for _ in range(256)]], ((0, seq_len-1), (0,0)), 'constant')
    else:
        mask = np.array([[1] for _ in range(seq_len)])
        embedding = np.array(embedding[0:seq_len])
    return embedding, mask

class ProductsDataset(Dataset):
    """Dataset wrapping tensors.

    Each sample will be retrieved by indexing tensors along the first dimension.

    Arguments:
        *tensors (Tensor): tensors that have the same size of the first dimension.
    """

    def __init__(self, *tensors):
#         assert all(tensors[0].size(0) == tensor.size(0) for tensor in tensors)
        self.tensors = tensors

    def __getitem__(self, index):
        distances = self.tensors[0][index]
        targets = self.tensors[1][index]
        overlap_freqs = str(self.tensors[2][index][0])
        unmatch_freqs = str(self.tensors[3][index][0])
        overlap_len = 12
        unmatch_len = 30
        if len(overlap_freqs) == 0:
            overlap_freqs = '0.0'
        if len(unmatch_freqs) == 0:
            unmatch_freqs = '0.0'
        overlap_array = str(overlap_freqs).split(';')[0:overlap_len]
        unmatch_array = str(unmatch_freqs).split(';')[0:unmatch_len]
        
        overlap = np.pad([float(x) for x in overlap_array], (0, overlap_len-len(overlap_array)), 'constant')
        unmatch = np.pad([float(x) for x in unmatch_array], (0, unmatch_len-len(unmatch_array)), 'constant')
        return tuple([torch.FloatTensor(distances), torch.FloatTensor(targets), torch.FloatTensor(overlap), torch.FloatTensor(unmatch)])

    def __len__(self):
        return len(self.tensors[0]) 

use_cuda = torch.cuda.is_available()

#     torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

# train_loader = torch.utils.data.DataLoader(
#         ProductsDataset(train_features, train_overlap_freqs, train_unmatch_freqs, train_targets), batch_size=1024, shuffle=True)
# test_loader = torch.utils.data.DataLoader(
#         ProductsDataset(test_features, test_overlap_freqs, test_unmatch_freqs, test_targets), batch_size=1024, shuffle=True)



# train_loader2 = torch.utils.data.DataLoader(
#         ProductsDataset2(), batch_size=128, shuffle=True)
# test_loader2 = torch.utils.data.DataLoader(
#         ProductsDataset2(), batch_size=128, shuffle=True)


In [ ]:

import argparse
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

class Model(nn.Module):
    
    def __init__(self, input_size):
        super(Model, self).__init__()
        
        self.linear1 = nn.Linear(input_size, 256)
        self.linear2 = nn.Linear(256, 128)
        
    def forward(self, features):
        hid = F.relu(self.linear1(features))
        hid = F.relu(self.linear2(hid))
        return hid
    
class Output(nn.Module):
    
    def __init__(self, input_len):
        super(Output, self).__init__()
        self.linear1 = nn.Linear(input_len, 256)
        self.linear3 = nn.Linear(256, 1)
        
    def forward(self, embeddings):
        inputs = torch.cat(embeddings, dim=1)
        hid = F.relu(self.linear1(inputs))
        hid = F.sigmoid(self.linear3(hid))
        return hid

def l1_loss(estimates, targets):
#     targets = targets.unsqueeze(1)
#     print (torch.cat((estimates, targets), 1))
    return F.mse_loss(estimates, targets)
#     return (-targets * (torch.log(estimates + 0.0001)) - (1.0 - targets) * (torch.log(1.0 - estimates + 0.0001))).mean()

def print_loss(estimates, targets):
    true_positive = 0
    false_positive = 0
    true_negative = 0
    false_negative = 0
    for i in range(len(estimates)):
        if estimates[i][0] >= 0.5 and targets[i] == 1:
            true_positive += 1
        elif estimates[i][0] < 0.5 and targets[i] == 1:
            true_negative += 1
        elif estimates[i][0] < 0.5 and targets[i] == 0:
            false_negative += 1
        else:
            false_positive += 1
    if true_positive + false_positive == 0:
        precision = 1
    else:
        precision = float(true_positive) / (true_positive + false_positive)
    if true_positive + true_negative == 0:
        recall = 1
    else:
        recall = float(true_positive) / (true_positive + true_negative)
    return precision, recall
        
# strctured_model = Model()
# unstructured_model = bilstm_attn(128, 256, 256, 30000, 256, True, 0.1, use_cuda, 512, 1000)
# title_model = bilstm_attn(128, 256, 256, 30000, 256, True, 0.1, use_cuda, 512, 20)
# output_model = Output()

# new_structured_model = Model()
# new_title_model = Attention(128, 256, 256, 30000, 256, True, 0.1, use_cuda, 512, 20)
# new_output_model = Output()

# torch.manual_seed(1)
# distance_model = Model(10)
# overlap_model = Model(12)
# unmatch_model = Model(30)
# output_model = Output(384)

# opt = optim.Adam([{'params':output_model.parameters()},{'params':distance_model.parameters()},{'params':overlap_model.parameters()},{'params':unmatch_model.parameters()}], lr=0.001)

for e in range(20):
    distance_model.train()
    overlap_model.train()
    unmatch_model.train()
    output_model.train()
    for batch_idx, (distances, targets, overlaps, unmatches) in enumerate(train_loader):
        distance = Variable(distances)
        target = Variable(targets)
        overlap = Variable(overlaps)
        unmatch = Variable(unmatches)
        
        opt.zero_grad()
        
        distance_embedding = distance_model(distance)
        overlap_embedding = overlap_model(overlap)
        unmatch_embedding = unmatch_model(unmatch)
        
        estimates = output_model([distance_embedding, overlap_embedding, unmatch_embedding])
        loss = l1_loss(estimates, targets)
        print('Training: Iteration {0}, Batch {1}, Loss {2}'.format(e, batch_idx, loss.item()))
        loss.backward()
        opt.step()
        
    distance_model.eval()
    overlap_model.eval()
    unmatch_model.eval()
    output_model.eval()
    for batch_idx, (distances, targets, overlaps, unmatches) in enumerate(test_loader):
        distance = Variable(distances)
        target = Variable(targets)
        overlap = Variable(overlaps)
        unmatch = Variable(unmatches)
        
        distance_embedding = distance_model(distance)
        overlap_embedding = overlap_model(overlap)
        unmatch_embedding = unmatch_model(unmatch)
        
        estimates = output_model([distance_embedding, overlap_embedding, unmatch_embedding])
        loss = l1_loss(estimates, targets)
        
        prec, rec = print_loss(estimates, targets)
        
        print('Testing: Batch {}, Loss {}, Precision {}, Recall {}'.format(batch_idx, loss.item(), prec, rec))

In [ ]:
test_pairs = candidate_pairs.sample(n=10000, random_state=7)
feature_extraction(test_pairs)


In [ ]:
data_loader = torch.utils.data.DataLoader(
        ProductsDataset(candidate_pairs.iloc[:, 4:14].values, candidate_pairs.iloc[:, 5:6].values, candidate_pairs.iloc[:, 14:15].values,candidate_pairs.iloc[:, 15:16].values), batch_size=1000, shuffle=False)

distance_model.eval()
overlap_model.eval()
unmatch_model.eval()
output_model.eval()

scores = []
matches = 0

for batch_idx, (distances, targets, overlaps, unmatches) in enumerate(data_loader):
    distance = Variable(distances)
    target = Variable(targets)
    overlap = Variable(overlaps)
    unmatch = Variable(unmatches)

    distance_embedding = distance_model(distance)
    overlap_embedding = overlap_model(overlap)
    unmatch_embedding = unmatch_model(unmatch)
    
    estimates = output_model([distance_embedding, overlap_embedding, unmatch_embedding])
    
    for i in range(estimates.shape[0]):
        predict = estimates[i].item()
        if predict >= 0.5:
            matches += 1
        scores.append(predict)
    print ('Matching Pairs: {}, Total Pairs: {}'.format(matches, batch_idx * 1000))
    
# ['scores'] = scores
    

In [ ]:
candidate_pairs['scores'] = scores

In [ ]:
candidate_pairs[candidate_pairs['scores'] >= 0.6][['left_spec_id', 'right_spec_id']].to_csv('submit_v8.csv', index=False)


In [ ]:
candidate_pairs[candidate_pairs['scores'] >= 0.999][['left_spec_id', 'right_spec_id']].to_csv('submit_v10.csv', index=False)


In [ ]:
# torch.save(strctured_model.state_dict(), 'structured.model')
# torch.save(title_model.state_dict(), 'title.model')
# torch.save(output_model.state_dict(), 'output.model')

# torch.save(distances_model.state_dict(), 'distances_model.model')
# torch.save(freqs_model.state_dict(), 'freqs_model.model')

torch.save(distance_model.state_dict(), 'distance_model.model')
torch.save(overlap_model.state_dict(), 'overlap_model.model')
torch.save(unmatch_model.state_dict(), 'unmatch_model.model')
torch.save(output_model.state_dict(), 'output_model.model')

In [ ]:
distance_model = Model(10)
overlap_model = Model(12)
unmatch_model = Model(30)
output_model = Output(384)

distance_model.load_state_dict(torch.load('distance_model.model'))
overlap_model.load_state_dict(torch.load('overlap_model.model'))
unmatch_model.load_state_dict(torch.load('unmatch_model.model'))
output_model.load_state_dict(torch.load('output_model.model'))



In [43]:
import pandas as pd
candidates = pd.read_csv('/home/sunji/EM_sigmod/quickstart_package/candidate_pairs_quickstart.csv')
# structured_data = pd.read_csv('total_with_key_type.csv')

In [90]:
# xxxx = pd.read_csv('/home/sunji/EM_sigmod/sigmod_large_labelled_dataset.csv')
# xxxx = pd.read_csv('/home/sunji/EM_sigmod/quickstart_package/output/submission.csv')
xxxx = pd.read_csv('/home/sunji/EM_sigmod/submit_v21.csv')
# xxxx.columns = ['left_spec_id', 'right_spec_id', 'left_spec_title', 'right_spec_title']

In [91]:
xxxx = xxxx.join(structured_data[['type', 'type_number', 'blocking_key', 'page_title']], on='left_spec_id', how='inner').join(structured_data[['type', 'type_number', 'blocking_key', 'page_title']], on='right_spec_id', how='inner', lsuffix='_left', rsuffix='_right')


In [86]:
xxxx.to_csv('/home/sunji/EM_sigmod/quickstart_package/label_pairs_with_key_type.csv', index=False)

In [574]:
xxxx.to_csv('/home/sunji/EM_sigmod/pairs_with_key_type.csv', index=False)

In [89]:
xxxx.to_csv('/home/sunji/EM_sigmod/quickstart_package/candidate_pairs_with_key_type.csv', index=False)

In [36]:
structured_data = structured_data.set_index('spec_id')

In [95]:
xxxx.sample(n=50)

,left_spec_id,right_spec_id,type_left,type_number_left,blocking_key_left,page_title_left,type_right,type_number_right,blocking_key_right,page_title_right
397086,www.ebay.com//24559,www.ebay.com//57941,s3500,3500,nikon,nikon coolpix s3500 20 1mp digital camera 7x o...,s3500,3500,nikon,untested nikon coolpix s3500 20 1 mp digital c...
233587,www.alibaba.com//27307,www.alibaba.com//26680,ds2cd2632fis,22632,hikvision,hikvision cctv 3mp ds-2cd2632f-is vf bullet ca...,ds2cd2632fis,22632,hikvision,3mp ir bullet ds-2cd2632f-is excellence hot ip...
201253,www.ebay.com//47470,www.ebay.com//42440,xp70,70,fuji,fujifilm finepix xp70 bundle | ebay,xp70,70,fuji,fujifilm finepix xp70 16 4 mp digital camera o...
7098,www.alibaba.com//26417,www.alibaba.com//27105,a20;two,20,NaN,7 inch 3g calling latest tablet reviews digita...,a20;two,20,NaN,7 inch 3g calling touch screen tablets digital...
357276,cammarkt.com//307,www.ebay.com//48337,d40x,40,nikon,cammarkt | nikon d40x 10.2mp digital slr camer...,d40x,40,nikon,nikon d40x digital camera body only parts repa...
540209,www.ebay.com//54498,www.ebay.com//60174,dscqx10,10,sony,sony smartphone attachable lens style camera d...,qx10,10,sony,sony cyber shot dsc qx10 18 2 mp digital camer...
428794,www.ebay.com//54175,www.ebay.com//56048,l11,11,nikon,nikon coolpix l11 6 0 mp digital camera brown ...,l11,11,nikon,digital camera nixon coolpix l11 6mp w 3x opti...
68457,www.ebay.com//47818,www.ebay.com//44967,1100d;t3,1100;3,cannon,canon eos rebel t3 1100d 12 2 mp digital slr c...,t3;s,3,cannon,new canon eos rebel t3 12 2 mp digital slr cam...
368528,www.ebay.com//45973,www.ebay.com//44676,j1;1,1;1,nikon,nikon 1 j1 10 1 mp digital camera white kit w ...,j1;two,1,nikon,nikon j1 two lens costco kit | ebay
394468,www.ebay.com//44852,www.ebay.com//56662,d3200,3200,nikon,nikon d3200 24 2 mp digital slr camera black w...,d3200,3200,nikon,nikon d3200 black bundle w 18 55mm 55 200mm ba...


In [83]:
xxxx[xxxx['type_left'].str.startswith('t3') | xxxx['type_right'].str.startswith('t3')]

,left_spec_id,right_spec_id,type_left,type_number_left,blocking_key_left,page_title_left,type_right,type_number_right,blocking_key_right,page_title_right
144822,www.ebay.com//56973,www.ebay.com//55226,t3;d,3,cannon,canon eos rebel t3 camera d slr kit ef s 18 55...,d;5,5,cannon,canon 5 d mark 2 013803105384 | ebay
72130,cammarkt.com//219,www.ebay.com//47700,t3i,3,cannon,cammarkt | canon eos rebel t3i 18 mp cmos aps-...,100d;t3,100;3,cannon,canon eos rebel t3 100d 12 2 mp digital slr ca...
72131,cammarkt.com//377,www.ebay.com//47700,t3two,3,cannon,"cammarkt | eos rebel t3 12mp slr camera, ef-s ...",100d;t3,100;3,cannon,canon eos rebel t3 100d 12 2 mp digital slr ca...
72132,cammarkt.com//328,www.ebay.com//47700,t3,3,cannon,cammarkt | canon eos rebel t3 12.2 mp cmos dig...,100d;t3,100;3,cannon,canon eos rebel t3 100d 12 2 mp digital slr ca...
72133,cammarkt.com//38,www.ebay.com//47700,t3,3,cannon,cammarkt | canon eos rebel t3 12.2 mp cmos dig...,100d;t3,100;3,cannon,canon eos rebel t3 100d 12 2 mp digital slr ca...
72134,cammarkt.com//460,www.ebay.com//47700,t3i,3,cannon,cammarkt | canon eos rebel t3i 18 mp cmos digi...,100d;t3,100;3,cannon,canon eos rebel t3 100d 12 2 mp digital slr ca...
72135,cammarkt.com//504,www.ebay.com//47700,t3i,3,cannon,cammarkt | canon eos rebel t3i 18 mp cmos aps-...,100d;t3,100;3,cannon,canon eos rebel t3 100d 12 2 mp digital slr ca...
72136,cammarkt.com//309,www.ebay.com//47700,t3,3,cannon,cammarkt | canon eos rebel t3 12.2 mp cmos dig...,100d;t3,100;3,cannon,canon eos rebel t3 100d 12 2 mp digital slr ca...
72137,cammarkt.com//262,www.ebay.com//47700,t3i,3,cannon,cammarkt | canon eos rebel t3i 18 mp cmos aps-...,100d;t3,100;3,cannon,canon eos rebel t3 100d 12 2 mp digital slr ca...
72138,www.gosale.com//749,www.ebay.com//47700,t3,3,cannon,canon eos rebel t3 on sale for $519.99,100d;t3,100;3,cannon,canon eos rebel t3 100d 12 2 mp digital slr ca...


In [99]:
import re
stopped_words = set(['ixus','pro','rebel','shot','cyber','catalog','stylus','mark','edition','elph','','digital camera', 'camera','wide','model','sport','network','action','full','ir','mini','metal','digital camera', 'indoor', 'canon eos', 'compact', 'digital', 'digital rebel', 'case', 'lcd', 'panasonic lumix', 'slr', 'cybershot', 'hd', '3x', '4x', '5x', '3gp', '']) | all_brands
def is_type(words, idx, current_num):
#     if len(words) > 3:
#         return True
#     print (words[idx], current_num)
    exist_number_match = re.findall(r'[0-9]+', words[idx])
    only_number_match = re.findall(r'^[0-9]+$', words[idx])
    possible_mpn_match = re.findall(r'\d{4}', words[idx])
    letter_match = re.findall(r'[a-z]+', words[idx])
    stop_match = re.findall(r'["$\']+', words[idx])
    if 'comparison' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
#     if 'buy' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
#         return False
    if 'kit' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if '&' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if '+' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if 'w/' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if 'w' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if 'with' in words[0:idx] and len(set(words[0:idx]) & all_brands) != 0: # Kit is not important
        return False
    if len(words[idx]) > 20:
        return False
    if not exist_number_match and words[idx] not in ['i','ii','iii','vi','iv','v','xs', 'k-x', 'k-r', 'nx', 'xs-pro', 'v']: # No number, cannot be type
        return False
    if only_number_match and idx == 0:
        return False
    if not letter_match:
        if stop_match: # money, size etc.
            return False
        if idx < len(words)-1 and (words[idx+1] in ['mm', 'mp', 'inch', 'megapixel', 'mega', 'pack']):
            return False
#         if not possible_mpn_match:
        if len(set(words[max(0, idx-2):idx]) & all_brands) == 0 and words[idx-1] not in ['rebel', 'finepix'] or (words[idx-2] == 'eos' and words[idx-1] != 'rebel') or words[idx-2] == 'finepix': # no adjacent brand
            if current_num >= 1:
                return False
            if idx < len(words)-1 and (words[idx+1].endswith('mm') or words[idx+1].endswith('mp') or words[idx+1].endswith('inch') or words[idx+1].endswith('megapixel') or words[idx+1].endswith('mega')):
                return False
            if only_number_match and idx < len(words)-2 and re.findall(r'^[0-9]+$', words[idx+1]):
                if float(words[idx+1]) < 10 and (words[idx+2] in ['mp', 'inch', 'mega', 'megapixel']): # still a pixel, missing point
                    return False
                if float(words[idx]) < 300 and float(words[idx+1]) < 1000 and words[idx+2] in ['mm', 'lens']: # still a len, missing point
                    return False
            if re.findall(r'[0-9]+\-[0-9]+', words[idx]):
                return False
        else:
            if only_number_match and idx < len(words)-2 and re.findall(r'^[0-9]+$', words[idx+1]):
                if float(words[idx+1]) < 10 and (words[idx+2] in ['mp', 'inch', 'mega', 'megapixel']): # still a pixel, missing point
                    return False
                if float(words[idx]) < 300 and float(words[idx+1]) < 1000 and words[idx+2] in ['mm', 'lens']: # still a len, missing point
                    return False
            if current_num >= 1 and idx < len(words)-1 and (words[idx+1].endswith('mm') or words[idx+1].endswith('mp') or words[idx+1].endswith('inch') or words[idx+1].endswith('megapixel') or words[idx+1].endswith('mega')):
                return False
        if words[idx] in ['2015', '2014', '2013', '2012', '2011']:
            return False
    else:
#         print (words[idx])
        if words[idx] in ['i', 'ii', 'iii', 'iv'] and 'af-s' in words[0:idx] or (idx < len(words)-1 and words[idx+1] == 'lens'):
            return False
        if (re.findall(r'\d*\.?\d*\-?\d*\.?\d*mm$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*mp$', words[idx]) or
           re.findall(r'\d+\.?\d*\-?\d*\.?\d*m$', words[idx]) or
           re.findall(r'\d+\.?\d*\-?\d*\.?\d*cm$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megpixel$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*cmos$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*pcs$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*pc$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*inch$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*gb$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*g$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*fps$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megapixel$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*mega$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megapixels$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*year$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*yr$', words[idx]) or
           re.findall(r'^s\d+\.?\d*\-\d+\.?\d*$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*g?hz$', words[idx]) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*lens$', words[idx]) or
           re.findall(r'ip\d*$', words[idx])):
            return False
        if words[idx] in ['1080p', '960p', '720p', '360p', 'h.264', 'p2p']:
            return False
        if words[idx].startswith('sel') or words[idx].startswith('sal'):
            return False
        if re.findall(r'[0-9]+\-[0-9]+\/[0-9]+-[0-9]+$', words[idx]):
            return False
        if re.findall(r'^f\/[0-9]+\.?[0-9]*', words[idx]):
            return False
        if re.findall(r'^f\/?[0-9]+\.?[0-9]*$', words[idx]) and len(set(words[max(0, idx-2):idx]) & all_brands) == 0:
            return False
        if re.findall(r'^f\/?[0-9]+\.?[0-9]*\-[0-9]+\.?[0-9]*', words[idx]):
            return False
        if stop_match:
            return False
        if re.findall(r'^[0-9]+\.?[0-9]*x$', words[idx]) and len(set(words[max(0, idx-2):idx]) & all_brands) == 0:
            return False
        
    return True

def verify_model(word, current_size):
#     print (word)
    if re.findall(r'\d+', word) and current_size > 0:
#         print ('here')
        return False
    if (re.findall(r'\d*\.?\d*\-?\d*\.?\d*mm$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*mp$', word) or
           re.findall(r'\d+\.?\d*\-?\d*\.?\d*m$', word) or
           re.findall(r'\d+\.?\d*\-?\d*\.?\d*cm$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megpixel$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*mega$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*cmos$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*pcs$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*pc$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*inch$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*gb$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*g$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*fps$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megapixel$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*megapixels$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*year$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*yr$', word) or
           re.findall(r'^s\d+\.?\d*\-\d+\.?\d*$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*g?hz$', word) or
           re.findall(r'^\d*\.?\d*\-?\d*\.?\d*lens$', word) or
           re.findall(r'ip\d*$', word)):
#         print ('here2')
        return False
    if word in ['1080p', '720p', '360p', 'h.264', 'p2p']:
        return False
    if word.startswith('sel') or word.startswith('sal'):
        return False
    if re.findall(r'[0-9]+\-[0-9]+\/[0-9]+-[0-9]+$', word):
        return False
    if re.findall(r'^f\/[0-9]+\.?[0-9]*', word):
        return False
    if re.findall(r'^f\/?[0-9]+\.?[0-9]*$', word):
        return False
    if re.findall(r'^f\/?[0-9]+\.?[0-9]*\-[0-9]+\.?[0-9]*', word):
        return False
    if re.findall(r'^[0-9]+\.?[0-9]*x$', word):
        return False
    return True

def tfidf(x):
#     print (x)
    if x in word_tfidf:
        return word_tfidf[x]
    else:
        return 0.0

def extract_type(row):
    words = re.split(r'[, ()~]', str(row['page_title']))
    words = [x.strip(',;-()') for x in words]
    types = []
    generations = {'i':'one','ii':'two','iii':'three','iv':'four'}
    keep_num = 2
    for idx, w in enumerate(words):
        if is_type(words, idx, len(re.findall(r'\d+', ''.join(types)))):
            if w in ['i','ii','iii','iv'] and len(types) == 1:
                for i in range(len(types)):
                    types[i] = types[i] + '-' + generations[w]
#             if re.findall(r'\d+', w) or w in ['i','ii','iii','vi','iv']:
            else:
                if re.findall(r'^\d+$', w):
                    print ('here: ', w)
                    if idx > 0 and words[idx-1] in all_brands:
                        types.append(w)
                    elif idx > 1 and words[idx-2] in all_brands:
                        types.append(words[idx-1] + '-' + w)
                    elif idx > 2 and words[idx-3] in all_brands:
                        types.append(words[idx-1] + '-' + w)
                        types.append(words[idx-2] + '-' + w)
                        keep_num += 1
                    elif idx > 3 and words[idx-4] in all_brands:
                        types.append(words[idx-1] + '-' + w)
                        types.append(words[idx-2] + '-' + w)
                        types.append(words[idx-3] + '-' + w)
                        keep_num += 2
                    else:
                        types.append(w)
                else:
                    types.append(w)
#             types = list(set(types) - (stopped_words - all_brands))
    
    print (types)
    if row['camera_model'] != 'unknown' and row['camera_model'] != 'nan' and not pd.isnull(row['camera_model']):
        models = re.split(r'[; ]', str(row['camera_model']))
        mmmmms = []
        for m in models:
            if verify_model(m, len(types)):
                mmmmms.append(m.strip('"\''))
        models = set(mmmmms) - stopped_words
#         models = sorted(re.split(r'[; ]', str(row['camera_model'])), key=lambda x: tfidf(x), reverse=False)
#         models = models[0:min(len(models), 2)]
        models = sorted(models, key=lambda x: tfidf(x), reverse=False)
        if len(models) > 0:
            types = set(types) | set(models[0:1])
    if len(types) == 0 and row['camera_mpn'] != 'unknown':
        return row['camera_mpn']
    types = set([''.join(t.split('-')) for t in types]) - stopped_words
    types = sorted(types, key=lambda x: tfidf(x), reverse=False)
#     types = types[0:min(len(types), 5)]
#     has_number = []
#     for i in range(len(types)):
#         if re.findall(r'\d+', types[i]):
#             has_number.append(i)
#     if len(has_number) > 0:
#         typess = []
#         for ii in has_number:
#             typess.append(types[ii])
#         types = typess
    types = types[0:min(len(types), keep_num)]
    return ';'.join(types)

def extract_type_number(row):
    all_types = str(row['type']).split(';')
    numbers = []
    for t in all_types:
        num = re.findall(r'\d+', str(t))
        if len(num) > 0:
            numbers.append(''.join(num))
    return ';'.join(numbers)
        

In [80]:
import swifter
structured_data['type'] = structured_data.swifter.apply(lambda row: extract_type(row), axis=1)
structured_data['type_number'] = structured_data.swifter.apply(lambda row: extract_type_number(row), axis=1)


In [101]:
print(extract_type(structured_data.loc['www.ebay.com//54284']))
structured_data.loc['www.ebay.com//54284']

[]
3


source                                                      www.ebay.com
spec_number                                                        54284
page_title             sony cyber shot 3 3 mega pixels camera set | ebay
blocking_key                                                        sony
type                                                                   3
camera_brand                                                        sony
camera_model                                           mega;cyber shot;3
camera_color                                                     unknown
camera_resolution                                                   0.11
camera_sensitivity                                                     0
camera_stablization                                              unknown
camera_battery                                                   unknown
camera_mpn                                                       unknown
camera_title           Sony Cyber Shot 3 3 Mega Pix

In [79]:
print(extract_type(structured_data.loc['www.eglobalcentral.co.uk//107']))
structured_data.loc['www.eglobalcentral.co.uk//107']['page_title']

tc20e


'nikon af-s teleconverter tc-20e iii'

In [26]:
structured_data.sample(n=20)

,source,spec_number,page_title,blocking_key,type,camera_brand,camera_model,camera_color,camera_resolution,camera_sensitivity,camera_stablization,camera_battery,camera_mpn,camera_title,type_number
spec_id,,,,,,,,,,,,,,,
www.canon-europe.com//131,www.canon-europe.com,131,canon powershot a2200 - powershot and ixus dig...,cannon,a2200,canon,a2200,unknown,0.000000,0.03125,unknown,Rechargeable Li-ion Battery NB-8L (battery and...,unknown,Canon PowerShot A2200 - PowerShot and IXUS dig...,2200
www.ebay.com//43013,www.ebay.com,43013,canon eos 70d dslr 20 2 mp canon ef s 18 135 e...,cannon,70d;s,eos;canon,135;canon eos;s;2;eos 70d;20;70d;300,unknown,0.673333,0.00000,unknown,unknown,unknown,Canon EOS 70D DSLR 20 2 MP Canon EF s 18 135 E...,70
www.alibaba.com//34175,www.alibaba.com,34175,"compact camera pouch,manfacturer for professio...",panasonic,dmcgf2,panasonic,panasonic lumix;dmc-gf2;lumix;digital camera;s...,unknown,0.000000,0.00000,unknown,unknown,unknown,"Compact Camera Pouch,Manfacturer For Professio...",2
www.ebay.com//44874,www.ebay.com,44874,panasonic dmc g5kk 16mp compact system camera ...,panasonic,g5kk,panasonic,compact;dmc-g5,unknown,0.533333,0.25000,unknown,unknown,DMC-G5KK,Panasonic DMC G5KK 16MP Compact System Camera ...,5
www.priceme.co.nz//2300,www.priceme.co.nz,2300,nikon d610 new zealand prices - priceme,nikon,d610,nikon,d610,unknown,0.768034,0.12500,Lens based only,unknown,unknown,Nikon D610 New Zealand Prices - PriceMe,610
www.ebay.com//60624,www.ebay.com,60624,used nikon d70 camera with af nikkor 35 70 mm ...,nikon,d70,nikon,d70;case;nikon d70,unknown,0.000000,0.00000,unknown,unknown,unknown,Used Nikon D70 Camera with AF Nikkor 35 70 mm ...,70
www.shopmania.in//945,www.shopmania.in,945,"kodak easyshare c143 digital camera prices, sh...",kodak,c143;c,kodak,c143;c,unknown,1.700000,0.03125,unknown,NiMH,unknown,"Kodak EasyShare C143 digital camera prices, Sh...",143
www.ebay.com//55970,www.ebay.com,55970,canon eos rebel xsi 12 2 mp kit w ef s 18 55mm...,cannon,450d,eos;canon,rebel xsi / 450d;canon eos;s;2;xsi;eos rebel;r...,unknown,0.406667,0.00000,unknown,unknown,2758B124AA,Canon EOS Rebel XSi 12 2 MP Kit w EF s 18 55mm...,450
www.ebay.com//48056,www.ebay.com,48056,sony alpha a230 body and everything shown 4905...,sony,a230,sony,a230;dslr-a230,unknown,0.340000,0.00000,unknown,unknown,DSLR-A230,Sony Alpha A230 Body and Everything Shown 4905...,230


In [321]:
structured_data.loc['www.alibaba.com//30151']['page_title']

'camera with ip address ds-2cd2632f-is hikvision ir network camera - buy camera with ip address,hikvision cctv camera,hikvision 3mp product on alibaba.com'

In [291]:
structured_data[structured_data['blocking_key'] == 'cannon'].sample(n=100)

,source,spec_number,page_title,blocking_key,type,camera_brand,camera_model,camera_color,camera_resolution,camera_sensitivity,camera_stablization,camera_battery,camera_mpn,camera_title
spec_id,,,,,,,,,,,,,,
www.ebay.com//24237,www.ebay.com,24237,canon eos 60d 18 0 mp digital slr camera black...,cannon,60d;2,eos;canon,canon eos;eos 60d;slr;2;60d,unknown,0.600000,0.000000,unknown,unknown,4460B040AA,Canon EOS 60D 18 0 MP Digital SLR Camera Black...
www.gosale.com//1318,www.gosale.com,1318,canon eos digital rebel xt 8mp digital on sale...,cannon,xt;rebel,eos;canon,canon eos;xt;slr;rebel xt;digital rebel,unknown,0.296667,0.000000,unknown,unknown,Digital Rebel X,Canon EOS Digital Rebel XT 8MP Digital on sale...
www.ebay.com//44793,www.ebay.com,44793,canon eos rebel t3i 600d 18 0 mp digital slr c...,cannon,t3i;600d;rebel,eos;canon,canon eos;slr;rebel t3i;rebel t3i / 600d;t3i;e...,unknown,0.600000,0.000000,unknown,unknown,5169B003,Canon EOS Rebel T3i 600D 18 0 MP Digital SLR C...
www.priceme.co.nz//1907,www.priceme.co.nz,1907,canon eos 600d + 18-55/3.5-5.6 is new zealand...,cannon,600d,eos;canon,eos 600d;canon eos,unknown,0.569531,0.125000,Lens based only,unknown,unknown,Canon EOS 600D + 18-55/3.5-5.6 IS New Zealand...
www.ebay.com//60595,www.ebay.com,60595,canon eos m2 twin lens kit ef 18 55mm m22mm is...,cannon,m2,eos;canon,eos m2;m2;canon eos,unknown,0.600000,0.000000,unknown,unknown,unknown,Canon EOS M2 Twin Lens Kit EF 18 55mm M22MM Is...
www.ebay.com//55197,www.ebay.com,55197,canon eos 1d mark iii 10.1mp digital slr camer...,cannon,1d-iii;1d;iii,eos;canon,canon eos;1d mark iii;slr;1d,unknown,0.336667,0.000000,unknown,unknown,EOS 1D Mark III,Canon EOS 1D Mark III 10.1MP Digital SLR Camer...
www.ebay.com//60078,www.ebay.com,60078,canon powershot digital elph 110 hs digital ix...,cannon,013803145649;110;125,canon,1;125 hs;digital elph 110 hs / digital ixus 12...,unknown,0.536667,0.000000,unknown,unknown,6039B001,Canon PowerShot Digital ELPH 110 HS Digital IX...
www.flipkart.com//2230,www.flipkart.com,2230,canon powershot ixus 145 point & shoot camera ...,cannon,rs.4660;145;ixus,canon,ixus 145;powershot,Black,0.066667,0.031250,unknown,unknown,unknown,Canon PowerShot IXUS 145 Point & Shoot Camera ...
www.shopmania.in//1248,www.shopmania.in,1248,canon powershot sx150 is digital camera prices...,cannon,sx150;150;is,canon,sx150 is;150,unknown,0.470000,0.031250,Optical,NiMH,unknown,Canon PowerShot SX150 IS digital camera prices...


In [281]:
structured_data.loc['www.ebay.com//43237']

source                                                      www.ebay.com
spec_number                                                        43237
page_title             perfect waterproof 2 inch tft lcd 8mp digital ...
blocking_key                                                         NaN
type                                                                   2
camera_brand                                           unbranded/generic
camera_model                                                       lcd;2
camera_color                                                     unknown
camera_resolution                                               0.266667
camera_sensitivity                                                     0
camera_stablization                                              unknown
camera_battery                                                   unknown
camera_mpn                                                       unknown
camera_title           Perfect Waterproof 2 inch TF

In [548]:
structured_data.to_csv('total_with_key_type.csv')

In [ ]:
structured_data['type'] = structured_data['type'].fillna('') 

In [ ]:
len(structured_data[pd.isnull(structured_data['type'])])

In [ ]:
import swifter
import re
import dask.dataframe as dd
import multiprocessing
import pandas as pd
import numpy as np
from dask.multiprocessing import get
import editdistance

# stopped_words = set(['digital camera', 'digital', 'camera'])
ddata = dd.from_pandas(candidate_pairs, npartitions=2*multiprocessing.cpu_count())
# ddata = dd.from_pandas(candidate_pairs[0:4], npartitions=2)
def apply_df(df):
#     structured_data = pd.read_csv('total_normalized.csv').set_index('camera_ids')
    print (len(df))
    return df.apply(lambda row: decide_keep(row, structured_data), axis=1)

def decide_keep(row, structured_data):
    stopped_words = set(['digital camera', 'camera']) | all_brands

    llll = structured_data.loc[row['left_spec_id']]
    rrrr = structured_data.loc[row['right_spec_id']]

    lset = set(str(llll['type']).split(';'))
    rset = set(str(rrrr['type']).split(';'))

    max_len = max(len(str(llll['type'])), len(str(rrrr['type'])))
    camera_type_edit_dis=(editdistance.eval(''.join(sorted(lset)), ''.join(sorted(rset))) / float(max_len))

    if len(lset) == 0 and len(rset) == 0:
        camera_type_jaccard_dis=(0.0)
        camera_type_overlap_dis=(0.0)
    else:
        camera_type_jaccard_dis=(1.0 - len(lset&rset) / float(len(lset|rset)))
        camera_type_overlap_dis=(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))

    lset = set(re.findall(r"\d+\.?\d*",str(llll['type'])))
    rset = set(re.findall(r"\d+\.?\d*",str(rrrr['type'])))

    if len(lset) == 0 and len(rset) == 0:
        camera_type_numeric_jaccard_dis=(0.0)
        camera_type_numeric_overlap_dis=(0.0)
    else:
        camera_type_numeric_jaccard_dis=(1-len(lset&rset) / float(len(lset|rset)))
        camera_type_numeric_overlap_dis=(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))

    lnumtitle = re.findall(r"\d+",str(llll['page_title']))
    rnumtitle = re.findall(r"\d+",str(rrrr['page_title']))

    lset = set(lnumtitle)
    rset = set(rnumtitle)

    if len(lset) == 0 and len(rset) == 0:
        camera_title_numeric_jaccard_dis=(0.0)
        camera_title_numeric_edit_dis=(0.0)
    else:
        camera_title_numeric_jaccard_dis=(1.0 - len(lset&rset) / float(len(lset|rset)))
        max_len = max(len(lnumtitle), len(rnumtitle))
        camera_title_numeric_edit_dis=(editdistance.eval(lnumtitle, rnumtitle) / float(max_len))

    ltitle = str(llll['page_title']).split(' ')
    rtitle = str(rrrr['page_title']).split(' ')
    lset = set(ltitle)
    rset = set(rtitle)

    if len(lset) == 0 and len(rset) == 0:
        camera_title_jaccard_dis=(0.0)
        camera_title_overlap_dis=(0.0)
        camera_title_edit_dis=(0.0)
    else:
        camera_title_jaccard_dis=(1.0-len(lset&rset) / float(len(lset|rset)))
        camera_title_overlap_dis=(1.0 - len(lset&rset) / float(max(len(lset), len(rset))))
        max_len = max(len(ltitle), len(rtitle))
        camera_title_edit_dis=(editdistance.eval(ltitle, rtitle) / float(max_len))
    overlap_freqs = []
    for w in lset&rset:
        if w in word_tfidf:
            overlap_freqs.append(str(word_tfidf[w]))
    camera_title_overlap_freq=(';'.join(overlap_freqs))
    unmatch_freqs = []
    for w in (lset|rset)-(lset&rset):
        if w in word_tfidf:
            unmatch_freqs.append(str(word_tfidf[w]))
    camera_title_unmatch_freq=(';'.join(unmatch_freqs))
        
    return [camera_type_jaccard_dis, camera_type_overlap_dis,
            camera_type_edit_dis, camera_type_numeric_jaccard_dis,
            camera_type_numeric_overlap_dis,camera_title_jaccard_dis,
            camera_title_overlap_dis,camera_title_edit_dis,
            camera_title_numeric_jaccard_dis,camera_title_numeric_edit_dis,
            camera_title_overlap_freq,camera_title_unmatch_freq]

print ('begin')
# column = {'camera_type_jaccard_dis':float,'camera_type_numeric_jaccard_dis':float,'camera_title_jaccard_dis':float,'camera_type_overlap_dis':float,'camera_type_numeric_overlap_dis':float,'camera_title_overlap_dis':float,'camera_title_overlap_freq':str}
# res = ddata.apply(lambda row: decide_keep(row, structured_data), axis=1, meta=pd.Series(dtype=float,name='x')).compute(scheduler='processes')
res = ddata.map_partitions(lambda df: apply_df(df), meta=pd.Series(name='x')).compute(scheduler='processes') 
# candidate_pairs['keep'] = candidate_pairs.swifter.apply(lambda row: decide_keep(), axis=1)


In [ ]:
pairs_df['camera_type_jaccard_dis'] = camera_type_jaccard_dis
    pairs_df['camera_type_overlap_dis'] = camera_type_overlap_dis
    pairs_df['camera_type_edit_dis'] = camera_type_edit_dis
    pairs_df['camera_type_numeric_jaccard_dis'] = camera_type_numeric_jaccard_dis
    pairs_df['camera_type_numeric_overlap_dis'] = camera_type_numeric_overlap_dis
    pairs_df['camera_title_jaccard_dis'] = camera_title_jaccard_dis
    pairs_df['camera_title_overlap_dis'] = camera_title_overlap_dis
    pairs_df['camera_title_edit_dis'] = camera_title_edit_dis
    pairs_df['camera_title_numeric_jaccard_dis'] = camera_title_numeric_jaccard_dis
    pairs_df['camera_title_numeric_edit_dis'] = camera_title_numeric_edit_dis
    pairs_df['camera_title_overlap_freq'] = camera_title_overlap_freq
    pairs_df['camera_title_unmatch_freq'] = camera_title_unmatch_freq

In [ ]:
res[10000]

In [ ]:

candidate_pairs['camera_type_jaccard_dis'] = [x[0] for x in res]
candidate_pairs['camera_type_overlap_dis'] = [x[1] for x in res]
candidate_pairs['camera_type_edit_dis'] = [x[2] for x in res]
candidate_pairs['camera_type_numeric_jaccard_dis'] = [x[3] for x in res]
candidate_pairs['camera_type_numeric_overlap_dis'] = [x[4] for x in res]
candidate_pairs['camera_title_jaccard_dis'] = [x[5] for x in res]
candidate_pairs['camera_title_overlap_dis'] = [x[6] for x in res]
candidate_pairs['camera_title_edit_dis'] = [x[7] for x in res]
candidate_pairs['camera_title_numeric_jaccard_dis'] = [x[8] for x in res]
candidate_pairs['camera_title_numeric_edit_dis'] = [x[9] for x in res]
candidate_pairs['camera_title_overlap_freq'] = [x[10] for x in res]
candidate_pairs['camera_title_unmatch_freq'] = [x[11] for x in res]


In [ ]:
candidate_pairs.loc[0:10]

In [ ]:
candidate_pairs.to_csv('candidates_with_distance.csv', index=False)

In [ ]:
candidate_pairs = pd.read_csv('candidates_with_distance.csv')

In [ ]:
candidate_pairs['keep'] = res

In [ ]:
candidate_pairs[candidate_pairs['keep'] == 1].to_csv('candidates_2.csv')

# Put them together

In [ ]:
candidate_pairs['camera_title_overlap_freq'] = candidate_pairs['camera_title_overlap_freq'].fillna('')

In [ ]:
# distances_data = list(candidate_pairs.iloc[:, 4:12].values)
# # freqs_data = list(candidate_pairs.iloc[:, 11:12].values)
# others_data = list(candidate_pairs.iloc[:, 4:5].values)

# data_loader = torch.utils.data.DataLoader(
#         ProductsDataset(distances_data, others_data), batch_size=1024, shuffle=False)

# distances_model.eval()
# freqs_model.eval()
output_model.eval()
results = []
matches = 0
start = time.time()
for batch_idx, (distances, targets) in enumerate(data_loader):
    distance = Variable(distances)
#     target = Variable(targets)
    freq = Variable(freqs)

#     distance_embedding = distances_model(distance) 
#     freq_embedding = freqs_model(freq)
    estimates = output_model(distance)
    
    for i in range(estimates.shape[0]):
        if estimates[i].item() >= 0.5:
            results.append(1)
            matches += 1
        else:
            results.append(0)
            
    end = time.time()
    print ('Testing: Iteration {}, Time: {}s, MatchNum: {}, CurrentNum: {}'.format(e, end-start, matches, len(results)))
    start = time.time()
# print ('Testing: Iteration {}'.format(e, np.mean(mse_errors), np.mean(precisions), np.mean(recalls)))


In [ ]:
len(results)

In [ ]:
candidate_pairs['predict'] = results

In [ ]:
candidate_pairs[candidate_pairs['predict'] == 1][['left_spec_id', 'right_spec_id']].to_csv('submit_v7.csv', index=False)


In [ ]:
candidate_pairs[candidate_pairs['predict'] == 1].to_csv('candidates_positive_with_distance.csv', index=False)

In [ ]:
len(candidate_pairs[candidate_pairs['predict'] == 0])